In [1]:
#this will not work because profit 1.1.5 has an issue where is fails. This also fails for versions 1.1.1, 1.1.2, 1.1.3, 1.1.4.

import pandas as pd
import yfinance as yf
from prophet import Prophet


c:\Users\hatfi\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [73]:
#get stock data
def get_stock_data(ticker: str, days: int) -> pd.DataFrame:
    """
    Fetches the Open, High, Low, Close, Adjusted Close, and Volume data for a given stock ticker and number of days.
    :param ticker: The stock ticker symbol.
    :param days: The number of days of historical data to fetch.
    :return: A pandas DataFrame containing the stock data.
    """
    # Calculate the start date based on the number of days
    end_date = pd.to_datetime('today')
    start_date = end_date - pd.Timedelta(days=days)
    
    # Fetch the data using yfinance
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    # Reset the index to add a default integer index
    stock_data.reset_index(inplace=True)
    
    return stock_data

# Example usage
ticker = "NVDA"
days = 3000
data = get_stock_data(ticker, days)

data


[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-03-28,0.860500,0.874500,0.859500,0.870750,0.853124,265388000
1,2016-03-29,0.867000,0.886750,0.865000,0.884750,0.866841,261812000
2,2016-03-30,0.888000,0.906250,0.887500,0.894000,0.875903,376700000
3,2016-03-31,0.898000,0.900000,0.883750,0.890750,0.872719,379884000
4,2016-04-01,0.885250,0.904250,0.878500,0.903750,0.885456,348292000
...,...,...,...,...,...,...,...
2062,2024-06-05,118.371002,122.448997,117.468002,122.440002,122.440002,528402000
2063,2024-06-06,124.047997,125.586998,118.320000,120.998001,120.998001,664696000
2064,2024-06-07,119.769997,121.692001,118.022003,120.888000,120.888000,412386000
2065,2024-06-10,120.370003,123.099998,117.010002,121.790001,121.790001,314162700


In [74]:
#Select the date and price
df = data[['Date', 'Close']]
df = df.rename(columns={'Date':'ds', 'Close':'y'})
df

,ds,y
0,2016-03-28,0.870750
1,2016-03-29,0.884750
2,2016-03-30,0.894000
3,2016-03-31,0.890750
4,2016-04-01,0.903750
...,...,...
2062,2024-06-05,122.440002
2063,2024-06-06,120.998001
2064,2024-06-07,120.888000
2065,2024-06-10,121.790001


In [75]:
#Get last 20 rows of data and store them into a new variable
last = df[len(df)-20:]
last

,ds,y
2047,2024-05-14,91.356003
2048,2024-05-15,94.629997
2049,2024-05-16,94.359001
2050,2024-05-17,92.478996
2051,2024-05-20,94.779999
2052,2024-05-21,95.386002
2053,2024-05-22,94.949997
2054,2024-05-23,103.799004
2055,2024-05-24,106.469002
2056,2024-05-28,113.901001


In [76]:
#Get all of the rows in the data set except for the last 20 rows of data
# df = df[:-20]
# df


In [77]:
# Create the Prophet object with daily seasonality
fbp = Prophet(daily_seasonality=True)

# Fit and train the model
fbp.fit(df)

# Make a future dataframe for 365 days
future = fbp.make_future_dataframe(periods=365)

# Predict the future
forecast = fbp.predict(future)


19:28:02 - cmdstanpy - INFO - Chain [1] start processing
19:28:04 - cmdstanpy - INFO - Chain [1] done processing
19:28:04 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
Optimization terminated abnormally. Falling back to Newton.
19:28:04 - cmdstanpy - INFO - Chain [1] start processing
19:28:04 - cmdstanpy - INFO - Chain [1] done processing
19:28:04 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 


RuntimeError: Error during optimization! Command 'C:\Users\hatfi\AppData\Local\Programs\Python\Python312\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=92017 data file=C:\Users\hatfi\AppData\Local\Temp\tmp461hkfrw\kjxniabm.json init=C:\Users\hatfi\AppData\Local\Temp\tmp461hkfrw\ueqjtcfr.json output file=C:\Users\hatfi\AppData\Local\Temp\tmp461hkfrw\prophet_modelltw184su\prophet_model-20240611192804.csv method=optimize algorithm=newton iter=10000' failed: 

In [ ]:
import pandas as pd
from prophet import Prophet

# Example data (replace this with your actual data)
data = {
    'ds': pd.date_range(start='2023-01-01', periods=30, freq='D'),  # 30 days of data
    'y': [10, 15, 14, 20, 25, 30, 35, 40, 20, 15, 10, 5, 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85]
}
df = pd.DataFrame(data)

# Ensure the 'ds' column is in datetime format
df['ds'] = pd.to_datetime(df['ds'])

# Print a summary of the data
print("Data Summary:")
print(df.info())
print(df.head())

# Check for missing values
if df.isnull().values.any():
    print("Data contains missing values. Please clean the data before proceeding.")
else:
    # Create the Prophet object with daily seasonality
    fbp = Prophet(daily_seasonality=True)

    # Fit and train the model
    try:
        fbp.fit(df)
    except RuntimeError as e:
        print(f"An error occurred during fitting: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"An unexpected error occurred: {ex}")
    else:
        # Make a future dataframe for 365 days
        future = fbp.make_future_dataframe(periods=365)

        # Predict the future
        forecast = fbp.predict(future)

        # Display the forecast
        print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])


19:14:12 - cmdstanpy - INFO - Chain [1] start processing


Data Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ds      30 non-null     datetime64[ns]
 1   y       30 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 612.0 bytes
None
          ds   y
0 2023-01-01  10
1 2023-01-02  15
2 2023-01-03  14
3 2023-01-04  20
4 2023-01-05  25


19:14:13 - cmdstanpy - INFO - Chain [1] done processing
19:14:13 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 


An error occurred during fitting: Error during optimization! Command 'C:\Users\hatfi\AppData\Local\Programs\Python\Python312\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=7941 data file=C:\Users\hatfi\AppData\Local\Temp\tmp461hkfrw\_x6_2e9v.json init=C:\Users\hatfi\AppData\Local\Temp\tmp461hkfrw\qrr84avs.json output file=C:\Users\hatfi\AppData\Local\Temp\tmp461hkfrw\prophet_modelobligi1w\prophet_model-20240611191412.csv method=optimize algorithm=newton iter=10000' failed: 
